# install ibkr api via pip3

[Setting Up Interactive Brokers API with Python](https://www.youtube.com/watch?v=0QcEweJdO1A)

In [ ]:
# list was is already installed
pip3 list 

# install latest ibkr api  => ibapi
pip3 install ibapi

In [ ]:
# connect local running TWS client - this instance should already login :-)

[code found here](https://www.pyquantnews.com/the-pyquant-newsletter/automate-trading-strategies-powerful-ib-api)

[2nd ](https://www.interactivebrokers.com/campus/ibkr-quant-news/interactive-brokers-python-api-native-a-step-by-step-guide/)


In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper  

class IBapi(EWrapper, EClient):
     def __init__(self):
         EClient.__init__(self, self) 

app = IBapi()

# paper
app.connect('127.0.0.1', 7496, 33)
app.run()
app.reqMarketDataType(3)


#Uncomment this section if unable to connect
#and to prevent errors on a reconnect
import time
time.sleep(20)
app.disconnect()


[app.reqMarketDataType(3)](https://www.interactivebrokers.com/campus/trading-lessons/python-receiving-market-data/)

[FROM HERE](https://www.interactivebrokers.com/campus/trading-lessons/python-receiving-market-data/)

In [ ]:
from ibapi.client import *
from ibapi.wrapper import *
import datetime
import time
import threading
from ibapi.ticktype import TickTypeEnum

# paper
port = 7496


class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId: OrderId):
        self.orderId = orderId
    
    def nextId(self):
        self.orderId += 1
        return self.orderId
    
    def error(self, reqId, errorCode, errorString, advancedOrderReject=""):
        print(f"reqId: {reqId}, errorCode: {errorCode}, errorString: {errorString}, orderReject: {advancedOrderReject}")
      
    def tickPrice(self, reqId, tickType, price, attrib):
        print(f"reqId: {reqId}, tickType: {TickTypeEnum.to_str(tickType)}, price: {price}, attrib: {attrib}")
      
    def tickSize(self, reqId, tickType, size):
        print(f"reqId: {reqId}, tickType: {TickTypeEnum.to_str(tickType)}, size: {size}")


app = TestApp()
app.connect("127.0.0.1", port, 0)
threading.Thread(target=app.run).start()
time.sleep(1)

mycontract = Contract()
mycontract.symbol = "AAPL"
mycontract.secType = "STK"
mycontract.exchange = "SMART"
mycontract.currency = "USD"

app.reqMarketDataType(3)
app.reqMktData(app.nextId(), mycontract, "232", False, False, [])

# portfolio

In [2]:
from ibapi.client import *
from ibapi.wrapper import *
import datetime
import time
import threading
from ibapi.ticktype import TickTypeEnum

# paper
port = 7496


class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)

    def nextValidId(self, orderId: OrderId):
        self.orderId = orderId
    
    def nextId(self):
        self.orderId += 1
        return self.orderId
    
    def error(self, reqId, errorCode, errorString, advancedOrderReject=""):
        print(f"reqId: {reqId}, errorCode: {errorCode}, errorString: {errorString}, orderReject: {advancedOrderReject}")
      
    def tickPrice(self, reqId, tickType, price, attrib):
        print(f"reqId: {reqId}, tickType: {TickTypeEnum.to_str(tickType)}, price: {price}, attrib: {attrib}")
      
    def tickSize(self, reqId, tickType, size):
        print(f"reqId: {reqId}, tickType: {TickTypeEnum.to_str(tickType)}, size: {size}")


app = TestApp()
app.connect("127.0.0.1", port, 0)
# threading.Thread(target=app.run).start()
time.sleep(3)

# mycontract = Contract()
# mycontract.symbol = "AAPL"
# mycontract.secType = "STK"
# mycontract.exchange = "SMART"
# mycontract.currency = "USD"

# app.reqMarketDataType(3)
# app.reqMktData(app.nextId(), mycontract, "232", False, False, [])



# get position two

In [ ]:
import threading
import pandas as pd

class ib_class(EWrapper, EClient): 
    def __init__(self): 
        EClient.__init__(self, self)
        self.all_positions = pd.DataFrame([], columns = ['Account','Symbol', 'Quantity', 'Average Cost'])
        self.execution_event = threading.Event()

    def position(self, account, contract, pos, avgCost):
        index = str(account)+str(contract.symbol)
        self.all_positions.loc[index]=account,contract.symbol,pos,avgCost
        # set the event
        self.positions_event.set()

    def error(self, reqId:TickerId, errorCode:int, errorString:str):
        if reqId > -1:
            print("Error. Id: " , reqId, " Code: " , errorCode , " Msg: " , errorString)
        # set the event
        self.positions_event.set()

    def positionEnd(self):
        super().positionEnd()
        self.disconnect()

ib_api = ib_class() 
ib_api.connect("127.0.0.1", 7496, 0) 

ib_api.positions_event.clear() # reset positions_event
ib_api.reqPositions()
ib_api.positions_event.wait()  # Wait for positions data to come in
current_positions = ib_api.all_positions
ib_api.run()

return(current_positions)

NameError: name 'pd' is not defined